## Installing uv

To start, you'll want to have uv installed:

https://docs.astral.sh/uv/getting-started/installation/


**Next, run this in the terminal, from the parent directory:**

    uv sync


Now you will have the kernel used to run this notebook

---

## Running the Required Servers

You need to run **three servers**, each in its own terminal:

- `central_server.py`
- `AS_server_twin.py`
- `Gatan_server_twin.py`

### 1. Activate the Virtual Environment

`uv` should have created a `.venv` directory for you.

**On macOS / Linux:**

    source .venv/bin/activate

**On Windows (likely):**

    source .venv/Scripts/activate

You should now see the environment activated.

---

### 2. Start the Servers

**Terminal 1 — Central Server**

    source .venv/bin/activate
    python -m asyncroscopy.servers.protocols.central_server

**Terminal 2 — AutoScript Server**

    source .venv/bin/activate
    python -m asyncroscopy.servers.AS_server_twin

**Terminal 3 — Gatan Twin Server**

    source .venv/bin/activate
    python -m asyncroscopy.servers.Gatan_server_twin

---

You're now ready to run this notebook! 🚀

In [ ]:
import sys
sys.path.insert(0, '../')
from asyncroscopy.clients.notebook_client import NotebookClient
import matplotlib.pyplot as plt

In [ ]:
# Connect the Client to the central (async) server
tem = NotebookClient.connect(host='localhost',port=9000)

# Tell the central server address of all connected instruments
routing_table= {"AS": ("localhost", 9001),
                "Gatan": ("localhost", 9002),
                "Ceos": ("localhost", 9003),
                "Preacquired_AS": ("localhost", 9004)}
tem.send_command('Central',"set_routing_table", routing_table)

In [ ]:
# connect to the AutoScript computer and initialize microscope
tem.send_command('AS',command='connect_AS',args={'host':'localhost','port':9001})


In [ ]:
stage = tem.send_command('AS',command='get_stage')
stage

In [ ]:
tem.send_command('AS',command='get_status') # if no args provided, uses {}

### Getting an image takes 5 seconds:

In [ ]:
image_args = {'scanning_detector':'HAADF',
                'size':512,
                'dwell_time':10e-6}

img = tem.send_command('AS','get_scanned_image', image_args)

plt.imshow(img, cmap="gray")
plt.title("Simulated STEM Image")
plt.show()


### Getting a spectrum takes 3 seconds:

In [ ]:
spec_args = {'size':512}

spec = tem.send_command('Gatan','get_spectrum', spec_args)
plt.plot(spec)
plt.title("Simulated Spectrum")
plt.show()

### Why not both at once?

In [ ]:
image, spec = tem.send_parallel_commands([('AS','get_scanned_image', image_args),
                                            ('Gatan','get_spectrum', spec_args)])

fig, ax = plt.subplots(1,2, figsize = (10,5))
ax[0].imshow(image, cmap="gray")
ax[1].plot(spec)
